In [1]:
import numpy as np
import keras
import numpy as np
from keras import layers
import random
import sys


Using TensorFlow backend.


In [2]:
def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)


In [3]:
def loadFile():
    text = open('paper.txt').read().lower()
    return text

In [4]:
text = loadFile()
maxlen = 60
step = 3
sentences = []
next_chars = []

In [5]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

('Number of sequences:', 27512)
('Unique characters:', 68)
Vectorization...


In [6]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1

In [7]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [8]:
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


In [10]:
model.fit(x, y, batch_size=128, epochs=30, verbose=1)

Epoch 1/30
27512/27512 [==============================] - 35s 1ms/step - loss: 1.8214
Epoch 2/30
27512/27512 [==============================] - 35s 1ms/step - loss: 1.5720
Epoch 3/30
23424/27512 [========================>.....] - ETA: 5s - loss: 1.3965

27512/27512 [==============================] - 35s 1ms/step - loss: 1.3927
Epoch 4/30
27512/27512 [==============================] - 35s 1ms/step - loss: 1.2724
Epoch 5/30
27512/27512 [==============================] - 35s 1ms/step - loss: 1.1753
Epoch 6/30
 4480/27512 [===>..........................] - ETA: 29s - loss: 1.0338

27512/27512 [==============================] - 35s 1ms/step - loss: 1.0940
Epoch 7/30
27512/27512 [==============================] - 35s 1ms/step - loss: 1.0213
Epoch 8/30
27512/27512 [==============================] - 36s 1ms/step - loss: 0.9714
Epoch 9/30
  640/27512 [..............................] - ETA: 35s - loss: 0.8514

27512/27512 [==============================] - 36s 1ms/step - loss: 0.9217
Epoch 10/30
27512/27512 [==============================] - 36s 1ms/step - loss: 0.8888
Epoch 11/30
26880/27512 [============================>.] - ETA: 0s - loss: 0.8537

27512/27512 [==============================] - 36s 1ms/step - loss: 0.8541
Epoch 12/30
27512/27512 [==============================] - 36s 1ms/step - loss: 0.8264
Epoch 13/30
27512/27512 [==============================] - 36s 1ms/step - loss: 0.8025
Epoch 14/30
 4992/27512 [====>.........................] - ETA: 28s - loss: 0.6999

...

27512/27512 [==============================] - 35s 1ms/step - loss: 0.7370
Epoch 18/30
27512/27512 [==============================] - 35s 1ms/step - loss: 0.7172
Epoch 19/30
27512/27512 [==============================] - 37s 1ms/step - loss: 0.7055
Epoch 20/30
 3840/27512 [===>..........................] - ETA: 31s - loss: 0.5975

27512/27512 [==============================] - 36s 1ms/step - loss: 0.6969
Epoch 21/30
27512/27512 [==============================] - 36s 1ms/step - loss: 0.6914
Epoch 22/30
27512/27512 [==============================] - 36s 1ms/step - loss: 0.6762
Epoch 23/30
  128/27512 [..............................] - ETA: 35s - loss: 0.6960

27512/27512 [==============================] - 36s 1ms/step - loss: 0.6670
Epoch 24/30
27512/27512 [==============================] - 37s 1ms/step - loss: 0.6539
Epoch 25/30
26752/27512 [============================>.] - ETA: 1s - loss: 0.6458

27512/27512 [==============================] - 37s 1ms/step - loss: 0.6476
Epoch 26/30
27512/27512 [==============================] - 37s 1ms/step - loss: 0.6410
Epoch 27/30
27512/27512 [==============================] - 37s 1ms/step - loss: 0.6279
Epoch 28/30
 4864/27512 [====>.........................] - ETA: 29s - loss: 0.5374

27512/27512 [==============================] - 36s 1ms/step - loss: 0.6221
Epoch 29/30
27512/27512 [==============================] - 36s 1ms/step - loss: 0.6213
Epoch 30/30
27512/27512 [==============================] - 38s 1ms/step - loss: 0.6029


In [15]:
start_index = random.randint(0, len(text) - maxlen - 1)
generated_text = text[start_index: start_index + maxlen]
print('--- Generating with seed: "' + generated_text + '"')
sys.stdout.write(generated_text)
for i in range(1000):
    sampled = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(generated_text):
        sampled[0, t, char_indices[char]] = 1.
    preds = model.predict(sampled, verbose=0)[0]
    next_index = sample(preds, 0.5)
    next_char = chars[next_index]
    generated_text += next_char
    generated_text = generated_text[1:]
    if(i%100==0):
        next_char+='\n'
    sys.stdout.write(next_char)

--- Generating with seed: "ally, the service discovery requests that are issued in one "
ally, the service discovery requests that are issued in one d
efininted events. an applies the services in mobile ng the service is available the a network l

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


y-dir
eltion of the network. if node recentive bether is component is and records the postinations (manet 
is aven alsowe the service discovery sthed in association rules mining algorithm. the services in ot
 deperame the service discovery requests with the service discovery in mobile communing iscunter
 of the paper wish discovery in manet. services available the approach is based on the service disco
very algorithm is ad hoc networks shorkes hovery when shoriet is is nourses and the service discover
y in mobile the service discovery requests the resiscins nodes in manet, as as the service. the prop
osed approach is broadcasts the service is available on the node receiving the service. the proposed
 approach we have by a nogess probist, the service discovery in mobile latenc. in this paper we have
 service discovery is mobile discovery is manet. anot a compunentires local conse the poperations. 